<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b343dc315791bbc1175a2a3d97ae1d34c695751afba347010bd930463295f5ce
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-08 15:41:35
-------------------
qualified stocks: 89
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  7.27 L
Current:  1.40 C
-------------------
Today PnL: 18.62 K (0.13%)
Current PnL: -18.19 L (-12.23%)
CY Booked + Current PnL: -8.33 L (-5.6%)
-------------------
Total profit:  2.75 L
Total loss:  -20.94 L
-------------------
Total Booked + Current PnL: 18.65 L (15.32%)
Total Booked PnL: 36.84 L (30.27%)
Curr Year Booked PnL: 9.86 L (7.05%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 78.90 L (56.42%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.73%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DIXON,18940.15,-28.02,72.0,H-MC,8.00,144048.0,29112.0,7476.0,0.85,25.33,5.19,31.83,81.0,3.89,1.09,45.24,X40N,ATH,IT
4,APOLLOHOSP,8285.00,-14.30,57.0,H-LC,10.47,171303.0,15126.0,10963.0,-0.23,9.69,6.40,16.71,22.0,1.38,1.29,28.81,X40N,BTT,HEALTHCARE
79,TTKPRESTIG,770.00,104.48,61.0,M-SC,6.19,88065.0,-12712.0,12805.0,-1.09,-12.61,14.54,0.09,245.0,-0.99,0.66,15.70,OX40N,NTT,DURABLES
64,SBIN,863.00,-13.25,48.0,M-LC,4.52,214332.0,12852.0,14360.0,0.27,6.38,6.70,13.51,60.0,0.89,1.62,19.78,XY25,NTT,BANKS
33,HEROMOTOCO,5949.07,2.71,80.0,H-MC,21.99,190505.0,39592.0,17717.0,1.49,26.23,9.30,37.97,93.0,2.23,1.44,58.18,AR,ATH,AUTO


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,43.54,63.0,M-MC,7.51,226357.0,1395.0,166259.0,1.69,0.62,73.45,74.53,192.0,0.01,1.71,31.42,XY24,BTT,STEEL
31,HAVELLS,2069.16,-0.65,59.0,H-MC,1.90,246355.0,-1376.0,76444.0,-0.21,-0.56,31.03,30.30,92.0,-0.02,1.86,12.66,X40,ATH,ELECTRICAL
36,ICICIGI,2260.25,-21.04,39.0,H-MC,3.94,135642.0,-491.0,31618.0,-0.40,-0.36,23.31,22.86,91.0,-0.02,1.02,12.21,X40,ATH,INSURANCE
53,PGHH,17973.08,-31.21,50.0,H-MC,4.88,199140.0,-1680.0,70456.0,-1.62,-0.84,35.38,34.25,80.0,-0.02,1.50,4.32,X40,ATH,FMCG
88,WIPRO,420.00,-14.25,39.0,M-LC,6.89,150362.0,-583.0,110035.0,-0.47,-0.39,73.18,72.51,53.0,-0.01,1.13,5.61,XR,NTT,IT


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HCLTECH,1943.91,-3.66,30.0,H-LC,10.09,220271.0,-21653.0,84914.0,-1.13,-8.95,38.55,26.15,8.0,-0.25,1.66,4.09,X40,ATH,IT
41,INFY,2275.00,-21.75,37.0,H-LC,10.89,305208.0,-7680.0,179371.0,-0.81,-2.45,58.77,54.87,3.0,-0.04,2.30,4.03,X40,BTT,IT
76,TCS,4998.00,-28.69,38.0,H-LC,14.30,283786.0,-62170.0,186022.0,-0.96,-17.97,65.55,35.80,1.0,-0.33,2.14,0.53,X40,BTT,IT
85,VBL,671.64,-15.98,38.0,H-LC,6.90,302444.0,-13398.0,126059.0,0.94,-4.24,41.68,35.67,5.0,-0.11,2.28,9.02,X40N,ATH,FMCG
82,UNITDSPR,1644.00,-14.94,42.0,H-LC,8.45,171072.0,-10687.0,45933.0,-1.23,-5.88,26.85,19.39,86.0,-0.23,1.29,1.79,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LTIM,7230.20,-7.51,44.0,H-LC,1.77,229005.0,-17146.0,96365.0,-2.05,-6.97,42.08,32.18,16.0,-0.18,1.73,26.57,X200,ATH,IT
52,NESTLEIND,1377.00,-9.75,56.0,H-LC,2.72,277922.0,12496.0,44301.0,-1.72,4.71,15.94,21.40,11.0,0.28,2.10,12.13,XY25,NTT,FMCG
43,ITC,452.00,-38.34,48.0,H-LC,3.14,199292.0,-846.0,21743.0,0.05,-0.42,10.91,10.44,4.0,-0.04,1.50,5.12,X40,NTT,FMCG
19,CIPLA,1795.00,-20.81,46.0,H-LC,5.72,211144.0,6644.0,34775.0,-0.79,3.25,16.47,20.25,10.0,0.19,1.59,12.27,X40N,BTT,PHARMA
85,VBL,671.64,-15.98,38.0,H-LC,6.90,302444.0,-13398.0,126059.0,0.94,-4.24,41.68,35.67,5.0,-0.11,2.28,9.02,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.0,-37.63,65.0,L-SC,38.71,93715.0,-19834.0,59884.0,9.54,-17.47,63.90,35.27,268.0,-0.33,0.71,129.37,XR,NTT,HOTELS
66,SHALBY,327.0,1094.58,78.0,M-SC,3.25,162398.0,-18745.0,64212.0,6.79,-10.35,39.54,25.10,235.0,-0.29,1.22,30.15,XY24,NTT,HEALTHCARE
55,RAJESHEXPO,518.0,1758.59,59.0,L-SC,8.28,50702.0,-86475.0,86569.0,5.82,-63.04,170.74,0.07,267.0,-1.00,0.38,26.10,OX40N,NTT,JEWELLERY
83,VAIBHAVGBL,521.0,48.76,58.0,H-SC,11.46,99091.0,-52900.0,127025.0,5.45,-34.80,128.19,48.77,125.0,-0.42,0.75,15.34,XR,NTT,JEWELLERY
84,VALIANTORG,838.0,-327.70,60.0,H-SC,9.03,96876.0,-36410.0,121008.0,4.99,-27.32,124.91,63.47,139.0,-0.30,0.73,62.39,XR,NTT,CHEMICALS


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2326.38,9.17,52.0,H-MC,3.31,188420.0,23750.0,44222.0,-3.05,14.42,23.47,41.28,89.0,0.54,1.42,23.56,X40N,ATH,AC
56,RAJOOENG,143.33,-36.44,29.0,H-SC,11.14,96220.0,-6280.0,47109.0,-2.98,-6.13,48.96,39.83,114.0,-0.13,0.73,0.00,AR,ATH,MISC
48,LAOPALA,464.00,108.64,43.0,H-SC,3.88,66405.0,-34441.0,60269.0,-2.27,-34.15,90.76,25.61,142.0,-0.57,0.50,23.12,AR,NTT,CERAMICS
49,LTIM,7230.20,-7.51,44.0,H-LC,1.77,229005.0,-17146.0,96365.0,-2.05,-6.97,42.08,32.18,16.0,-0.18,1.73,26.57,X200,ATH,IT
63,SATIN,274.00,-27.73,53.0,H-SC,8.15,142382.0,-53592.0,126407.0,-1.95,-27.35,88.78,37.16,126.0,-0.42,1.07,8.25,XY24,NTT,FINANCE


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,104.48,61.0,M-SC,6.19,88065.0,-12712.0,12805.0,-1.09,-12.61,14.54,0.09,245.0,-0.99,0.66,15.70,OX40N,NTT,DURABLES
80,UJJIVANSFB,60.0,89.03,54.0,H-SC,12.94,119259.0,-23220.0,42742.0,3.76,-16.30,35.84,13.70,163.0,-0.54,0.90,40.76,OX40N,NTT,BANKS
46,KANSAINER,340.0,-68.16,55.0,H-SC,2.88,218979.0,-50688.0,87022.0,0.18,-18.80,39.74,13.47,138.0,-0.58,1.65,10.76,XY24,NTT,PAINTS
39,INDIGOPNTS,1408.0,115.96,51.0,M-SC,3.53,141558.0,-33001.0,33040.0,-0.33,-18.91,23.34,0.02,221.0,-1.00,1.07,23.13,OX40N,NTT,PAINTS
18,CERA,9475.0,-19.57,48.0,H-SC,2.52,114921.0,-28882.0,55633.0,0.69,-20.08,48.41,18.60,149.0,-0.52,0.87,26.82,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4998.00,-28.69,38.0,H-LC,14.30,283786.0,-62170.0,186022.0,-0.96,-17.97,65.55,35.80,1.0,-0.33,2.14,0.53,X40,BTT,IT
41,INFY,2275.00,-21.75,37.0,H-LC,10.89,305208.0,-7680.0,179371.0,-0.81,-2.45,58.77,54.87,3.0,-0.04,2.30,4.03,X40,BTT,IT
43,ITC,452.00,-38.34,48.0,H-LC,3.14,199292.0,-846.0,21743.0,0.05,-0.42,10.91,10.44,4.0,-0.04,1.50,5.12,X40,NTT,FMCG
85,VBL,671.64,-15.98,38.0,H-LC,6.90,302444.0,-13398.0,126059.0,0.94,-4.24,41.68,35.67,5.0,-0.11,2.28,9.02,X40N,ATH,FMCG
1,ABB,7934.00,-41.33,43.0,H-LC,12.93,245198.0,-16421.0,135644.0,-0.18,-6.28,55.32,45.57,7.0,-0.12,1.85,4.33,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,2.71,80.0,H-MC,21.99,190505.0,39592.0,17717.0,1.49,26.23,9.30,37.97,93.0,2.23,1.44,58.18,AR,ATH,AUTO
25,DIXON,18940.15,-28.02,72.0,H-MC,8.00,144048.0,29112.0,7476.0,0.85,25.33,5.19,31.83,81.0,3.89,1.09,45.24,X40N,ATH,IT
26,DMART,5201.00,-7.52,61.0,H-LC,16.03,230996.0,46515.0,23862.0,-1.65,25.21,10.33,38.14,38.0,1.95,1.74,38.49,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,43.54,63.0,M-MC,7.51,226357.0,1395.0,166259.0,1.69,0.62,73.45,74.53,192.0,0.01,1.71,31.42,XY24,BTT,STEEL
14,BLUESTARCO,2326.38,9.17,52.0,H-MC,3.31,188420.0,23750.0,44222.0,-3.05,14.42,23.47,41.28,89.0,0.54,1.42,23.56,X40N,ATH,AC
33,HEROMOTOCO,5949.07,2.71,80.0,H-MC,21.99,190505.0,39592.0,17717.0,1.49,26.23,9.30,37.97,93.0,2.23,1.44,58.18,AR,ATH,AUTO
86,VOLTAS,1918.49,1.44,61.0,H-MC,3.69,212880.0,21138.0,74891.0,-0.95,11.02,35.18,50.08,99.0,0.28,1.61,18.57,XY25,ATH,AC
23,DABUR,735.00,-2.22,63.0,H-MC,4.81,213018.0,15990.0,73640.0,-0.09,8.12,34.57,45.49,102.0,0.22,1.61,22.21,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,2.71,80.0,H-MC,21.99,190505.0,39592.0,17717.0,1.49,26.23,9.30,37.97,93.0,2.23,1.44,58.18,AR,ATH,AUTO
51,NATIONALUM,247.44,-43.66,71.0,H-MC,1.19,111597.0,11093.0,19552.0,-0.74,11.04,17.52,30.49,79.0,0.57,0.84,47.02,MH,ATH,METALS
87,WHIRLPOOL,2270.00,-39.53,62.0,M-SC,3.67,102225.0,10726.0,68021.0,0.44,11.72,66.54,86.07,223.0,0.16,0.77,48.45,XR,NTT,DURABLES
38,INDIAMART,4850.92,-51.09,49.0,H-SC,12.10,136454.0,13118.0,120639.0,0.76,10.64,88.41,108.45,119.0,0.11,1.03,35.37,AR,ATH,MISC
88,WIPRO,420.00,-14.25,39.0,M-LC,6.89,150362.0,-583.0,110035.0,-0.47,-0.39,73.18,72.51,53.0,-0.01,1.13,5.61,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,4546.37,-6.02,41.0,H-SC,15.41,87526.0,-13437.0,107963.0,-1.84,-13.31,123.35,93.63,148.0,-0.12,0.66,30.37,SR,ATH,CHEMICALS
50,MASFIN,398.61,-18.25,51.0,H-SC,12.87,93105.0,-4875.0,26479.0,-0.13,-4.98,28.44,22.05,152.0,-0.18,0.70,35.33,XR,ATH,FINANCE
21,COFFEEDAY,80.00,-37.63,65.0,L-SC,38.71,93715.0,-19834.0,59884.0,9.54,-17.47,63.90,35.27,268.0,-0.33,0.71,129.37,XR,NTT,HOTELS
84,VALIANTORG,838.00,-327.70,60.0,H-SC,9.03,96876.0,-36410.0,121008.0,4.99,-27.32,124.91,63.47,139.0,-0.30,0.73,62.39,XR,NTT,CHEMICALS
51,NATIONALUM,247.44,-43.66,71.0,H-MC,1.19,111597.0,11093.0,19552.0,-0.74,11.04,17.52,30.49,79.0,0.57,0.84,47.02,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-37.63,65.0,L-SC,38.71,93715.0,-19834.0,59884.0,9.54,-17.47,63.90,35.27,268.0,-0.33,0.71,129.37,XR,NTT,HOTELS
7,ATULAUTO,844.00,3954.05,77.0,M-SC,8.61,126307.0,-21061.0,72879.0,-0.66,-14.29,57.70,35.16,236.0,-0.29,0.95,29.01,XY24,NTT,AUTO
33,HEROMOTOCO,5949.07,2.71,80.0,H-MC,21.99,190505.0,39592.0,17717.0,1.49,26.23,9.30,37.97,93.0,2.23,1.44,58.18,AR,ATH,AUTO
62,SAMMAANCAP,326.00,-171.59,66.0,M-SC,5.41,82632.0,-19578.0,112966.0,-0.05,-19.15,136.71,91.37,208.0,-0.17,0.62,34.14,XY25,NTT,FINANCE
66,SHALBY,327.00,1094.58,78.0,M-SC,3.25,162398.0,-18745.0,64212.0,6.79,-10.35,39.54,25.10,235.0,-0.29,1.22,30.15,XY24,NTT,HEALTHCARE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.68
1,25,43.51
2,50,73.75


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.26
LC    35.92
MC    25.79
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.09
X40      15.44
X40N     14.18
XY25     10.06
AR        9.50
XR        9.19
OX40N     7.50
MH        1.76
X200      1.73
X5K       1.43
SR        1.09
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.59
H-MC    22.75
H-SC    22.70
M-SC    13.91
M-LC     7.25
M-MC     2.72
L-SC     1.65
L-LC     1.08
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.76,-1.06,35.75
IT,13.29,-17.09,75.14
BANKS,7.43,-15.68,64.80
FINANCE,7.41,-22.58,67.77
PAINTS,6.05,-15.15,40.93
MISC,5.84,-18.60,86.59
ELECTRICAL,5.70,-8.97,69.03
AUTO,4.96,-2.81,47.54
AC,3.76,2.89,30.00


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2738601.0,22
AR,1220189.0,10
XR,1074269.0,13
X40,918420.0,10
X40N,551893.0,11
OX40N,463326.0,10
XY25,419178.0,7
SR,198374.0,2
X5K,128168.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2763880.0,27
H-LC,1460890.0,16
M-SC,1398279.0,17
H-MC,1188581.0,17
M-LC,417713.0,5
M-MC,317203.0,2
L-SC,243015.0,3
L-MC,59807.0,1
L-LC,41043.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,930233.0,6
M-SC,XY24,813709.0,7
H-SC,AR,665331.0,5
H-LC,X40,623337.0,5
H-SC,XR,513018.0,6
H-MC,XY24,459170.0,4
H-LC,AR,378052.0,2
M-MC,XY24,317203.0,2
H-LC,X40N,241592.0,5


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
